In [ ]:
import pandas as pd 
import numpy as np
import os 
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy import stats

In [ ]:
files = os.listdir('/kaggle/input/network-scenario/Train_data(1)/Train_data')

In [ ]:
train_sets = [pd.read_csv('/kaggle/input/network-scenario/Train_data(1)/Train_data/'+i) for i in files]

# test data 
test_file = os.listdir('/kaggle/input/network-scenario/Test_data/Test_data')
test_sets = [pd.read_csv('/kaggle/input/network-scenario/Test_data/Test_data/'+i) for i in test_file]

In [ ]:
for i  in range(test_file.__len__()) :
    test_file[i] = test_file[i].removesuffix('.csv')

In [ ]:
#feature engineering for the testing data that requires to trait every network scenario indevualy 
for i in range(test_sets.__len__()):
    #lag features
    test_sets[i]['prtpktin_lag1'] = test_sets[i]['portPktIn'].shift(1)
    test_sets[i]['prtpktout_lag1'] = test_sets[i]['portPktOut'].shift(1)
    test_sets[i]['qSize_lag'] = test_sets[i]['qSize'].shift(1)
    
    test_sets[i]['prtpktin_lag2'] = test_sets[i]['portPktIn'].shift(2)
    test_sets[i]['prtpktout_lag2'] = test_sets[i]['portPktOut'].shift(2)
    test_sets[i]['qSize_lag2'] = test_sets[i]['qSize'].shift(2)
    
    test_sets[i]['prtpktin_lag3'] = test_sets[i]['portPktIn'].shift(3)
    test_sets[i]['prtpktout_lag3'] = test_sets[i]['portPktOut'].shift(3)
    test_sets[i]['qSize_lag3']= test_sets[i]['qSize'].shift(3)
    
    test_sets[i]['prtpktin_lag4'] = test_sets[i]['portPktIn'].shift(4)
    test_sets[i]['prtpktout_lag4'] = test_sets[i]['portPktOut'].shift(4)
    test_sets[i]['qSize_lag4']= test_sets[i]['qSize'].shift(4)
    
    test_sets[i]['prtpktin_lag5'] = test_sets[i]['portPktIn'].shift(5)
    test_sets[i]['prtpktout_lag5'] = test_sets[i]['portPktOut'].shift(5)
    test_sets[i]['qSize_lag5']= test_sets[i]['qSize'].shift(5)
    
    #future features
    test_sets[i]['prtpktin_forword'] = test_sets[i]['portPktIn'].shift(-1)
    test_sets[i]['prtpktout_forword'] = test_sets[i]['portPktOut'].shift(-1)
    test_sets[i]['qSize_forword'] = test_sets[i]['qSize'].shift(-1)
    
    test_sets[i]['prtpktin_forword2'] = test_sets[i]['portPktIn'].shift(-2)
    test_sets[i]['prtpktout_forword2'] = test_sets[i]['portPktOut'].shift(-2)
    test_sets[i]['qSize_forword2'] = test_sets[i]['qSize'].shift(-2)
    
    test_sets[i]['prtpktin_forword3'] = test_sets[i]['portPktIn'].shift(-3)
    test_sets[i]['prtpktout_forword3'] = test_sets[i]['portPktOut'].shift(-3)
    test_sets[i]['qSize_forword3'] = test_sets[i]['qSize'].shift(-3)
    
    test_sets[i]['prtpktin_forword4'] = test_sets[i]['portPktIn'].shift(-4)
    test_sets[i]['prtpktout_forword4'] = test_sets[i]['portPktOut'].shift(-4)
    test_sets[i]['qSize_forword4'] = test_sets[i]['qSize'].shift(-4)
    
    test_sets[i]['prtpktin_forword5'] = test_sets[i]['portPktIn'].shift(-5)
    test_sets[i]['prtpktout_forword5'] = test_sets[i]['portPktOut'].shift(-5)
    test_sets[i]['qSize_forword5'] = test_sets[i]['qSize'].shift(-5)
    
    
    test_sets[i]['group'] = test_file[i] + '_'  + test_sets[i]['time'].apply(lambda x:str(x))
    test_sets[i]['time_in_sec'] = test_sets[i]['time'].apply(lambda x: (x/1000).__trunc__()) 
    #aggregation
    agg = np.array(test_sets[i].drop(columns = 'group').groupby('time_in_sec').aggregate('mean')['portPktIn'])
    test_sets[i]['in_ts'] = agg[test_sets[i]['time_in_sec']]
    
    agg = np.array(test_sets[i].drop(columns = 'group').groupby('time_in_sec').aggregate('mean')['portPktOut'])
    test_sets[i]['out_ts'] = agg[test_sets[i]['time_in_sec']]
    
    agg = np.array(test_sets[i].drop(columns = 'group').groupby('time_in_sec').aggregate('mean')['qSize'])
    test_sets[i]['qSize_agg'] = agg[test_sets[i]['time_in_sec']]
    


In [ ]:
#feature engineering for the training data that requires to trait every network scenario indevualy 

for i in range(train_sets.__len__()):
    train_sets[i]['prtpktin_lag1'] = train_sets[i]['portPktIn'].shift(1)
    train_sets[i]['prtpktout_lag1'] = train_sets[i]['portPktOut'].shift(1)
    train_sets[i]['qSize_lag'] = train_sets[i]['qSize'].shift(1)
    
    train_sets[i]['prtpktin_lag2'] = train_sets[i]['portPktIn'].shift(2)
    train_sets[i]['prtpktout_lag2'] = train_sets[i]['portPktOut'].shift(2)
    train_sets[i]['qSize_lag2'] = train_sets[i]['qSize'].shift(2)
    
    train_sets[i]['prtpktin_lag3'] = train_sets[i]['portPktIn'].shift(3)
    train_sets[i]['prtpktout_lag3'] = train_sets[i]['portPktOut'].shift(3)
    train_sets[i]['qSize_lag3'] = train_sets[i]['qSize'].shift(3)
    
    train_sets[i]['prtpktin_lag4'] = train_sets[i]['portPktIn'].shift(4)
    train_sets[i]['prtpktout_lag4'] = train_sets[i]['portPktOut'].shift(4)
    train_sets[i]['qSize_lag4'] = train_sets[i]['qSize'].shift(4)
    
    train_sets[i]['prtpktin_lag5'] = train_sets[i]['portPktIn'].shift(5)
    train_sets[i]['prtpktout_lag5'] = train_sets[i]['portPktOut'].shift(5)
    train_sets[i]['qSize_lag5'] = train_sets[i]['qSize'].shift(5)
    
    train_sets[i]['prtpktin_forword'] = train_sets[i]['portPktIn'].shift(-1)
    train_sets[i]['prtpktout_forword'] = train_sets[i]['portPktOut'].shift(-1)
    train_sets[i]['qSize_forword'] = train_sets[i]['qSize'].shift(-1)
    
    train_sets[i]['prtpktin_forword2'] = train_sets[i]['portPktIn'].shift(-2)
    train_sets[i]['prtpktout_forword2'] = train_sets[i]['portPktOut'].shift(-2)
    train_sets[i]['qSize_forword2'] = train_sets[i]['qSize'].shift(-2)
    
    train_sets[i]['prtpktin_forword3'] = train_sets[i]['portPktIn'].shift(-3)
    train_sets[i]['prtpktout_forword3'] = train_sets[i]['portPktOut'].shift(-3)
    train_sets[i]['qSize_forword3'] = train_sets[i]['qSize'].shift(-3)
    
    train_sets[i]['prtpktin_forword4'] = train_sets[i]['portPktIn'].shift(-4)
    train_sets[i]['prtpktout_forword4'] = train_sets[i]['portPktOut'].shift(-4)
    train_sets[i]['qSize_forword4'] = train_sets[i]['qSize'].shift(-4)
    
    train_sets[i]['prtpktin_forword5'] = train_sets[i]['portPktIn'].shift(-5)
    train_sets[i]['prtpktout_forword5'] = train_sets[i]['portPktOut'].shift(-5)
    train_sets[i]['qSize_forword5'] = train_sets[i]['qSize'].shift(-5)
    
    
    train_sets[i]['time_in_sec'] = train_sets[i]['time'].apply(lambda x: (x/1000).__trunc__())  
    agg = np.array(train_sets[i].groupby('time_in_sec').aggregate('mean')['portPktIn'])
    train_sets[i]['in_ts'] = agg[train_sets[i]['time_in_sec']]
    
    agg = np.array(train_sets[i].groupby('time_in_sec').aggregate('mean')['portPktOut'])
    train_sets[i]['out_ts'] = agg[train_sets[i]['time_in_sec']]
    
    agg = np.array(train_sets[i].groupby('time_in_sec').aggregate('mean')['qSize'])
    train_sets[i]['qSize_agg'] = agg[train_sets[i]['time_in_sec']]
    


In [ ]:
train = pd.concat(train_sets)
test = pd.concat(test_sets)


In [ ]:
train['in_out_mean'] = (train['portPktIn'] + train['portPktOut'])/2
test['in_out_mean'] = (test['portPktIn'] + test['portPktOut'])/2

train['in_out_ts_mean'] = (train['in_ts']+train['out_ts'])/2
test['in_out_ts_mean'] = (test['in_ts']+test['out_ts'])/2

In [ ]:
id = test['group']
id = id.apply(lambda x:x.lower())


In [ ]:
train

In [ ]:
test.drop(columns=['group'],inplace=True)

In [ ]:
test

In [ ]:
train

In [ ]:
y = train['label']

In [ ]:
train.drop(columns=['label'],inplace=True)


In [ ]:
train

In [ ]:
test

In [ ]:
train.drop(columns ='time',inplace=True)
test.drop(columns ='time', inplace=True)

In [ ]:
lgb = LGBMClassifier(random_state=42)
lgb.fit(X=train,y=y)


In [ ]:
prediction = lgb.predict(test)

In [ ]:
submition = pd.DataFrame({
    'ID':id,
    'Target':prediction
})

In [ ]:
submition.set_index('ID',inplace=True)

In [ ]:
submition.to_csv('/kaggle/working/submission-cv87.csv')

In [ ]:
submition